In [8]:
from collections import defaultdict
import string
from pprint import pprint

In [2]:
class SafeDict(dict):
     def __missing__(self, key):
         return '{' + key + '}'

In [3]:
class JobFormatter(string.Formatter):
    def __init__(self, default='{0}'):
        self.default=default

    def get_value(self, key, args, kwds):
        if isinstance(key, str):
            return kwds.get(key, self.default.format(key))
        else:
            Formatter.get_value(key, args, kwds)

In [4]:
vals = SafeDict({'slurm_account': 'eros',
         'slurm_partition': 'normal',
         'slurm_log_out': '/stuff',
         'slurm_log_error': '/err_stuff',
         'slurm_email': 'me',
         'slurm_job_input_dir': '/here',
         'change_detection_path': '/there'})

In [6]:
vals

{'change_detection_path': '/there',
 'slurm_account': 'eros',
 'slurm_email': 'me',
 'slurm_job_input_dir': '/here',
 'slurm_log_error': '/err_stuff',
 'slurm_log_out': '/stuff',
 'slurm_partition': 'normal'}

In [5]:
stuff = {'slurm_account': 'eros',
         'slurm_partition': 'normal',
         'slurm_log_out': '/stuff',
         'slurm_log_error': '/err_stuff',
         'slurm_email': 'me',
         'slurm_job_input_dir': '/here',
         'change_detection_path': '/there'}

In [6]:
p = r'D:\projects\CCDC_Processing\CCDC_Processing\change_detection\submit_job_template.slurm'
with open(p, 'r') as f:
    temp = f.read()

In [9]:
#temp2 = temp.format(**vals)
pprint(string.Formatter().vformat(temp, (), vals))

'#!/bin/csh\n\n#SBATCH -A eros\n#SBATCH -p normal\n#SBATCH -t 4-12:00:00\n#SBATCH -n 1\n#SBATCH -o /stuff/change_detection_%a.out\n#SBATCH -e /err_stuff/change_detection_%a.error\n#SBATCH --open-mode=append\n#SBATCH --job-name=ccdc\n#SBATCH --array=1-5000\n#SBATCH --mail-type=ALL\n#SBATCH --mail-user=me\n#SBATCH --mem-per-cpu=24576\n\nsleep `shuf -i 1-120 -n 1`\n\nmodule load matlab/mcr-R2013a-v81\n\necho "\\n\\n********************************************"\necho "* This job runs on ${SLURM_JOB_NODELIST}"\necho "********************************************\\n\\n"\n\n# Set this to the last number of the array. New variable in Slurm 15.08 (coming soon)\nsetenv MATLAB_NTASKS 5000\n#setenv MATLAB_NTASKS ${SLURM_ARRAY_TASK_MAX}\nsetenv MATLAB_TASK ${SLURM_ARRAY_TASK_ID}\n\necho "MATLAB_NTASKS = ${MATLAB_NTASKS}"\necho "MATLAB_TASK = ${MATLAB_TASK}"\n\n\n# Running MATLAB standalone results in creation of a cache folder in home dir\n# This may cause runtime issues. Below is a workaround.\nech

In [11]:
JobFormatter().format(temp, vals)

'#!/bin/csh\n\n#SBATCH -A {slurm_account}\n#SBATCH -p {slurm_partition}\n#SBATCH -t 4-12:00:00\n#SBATCH -n 1\n#SBATCH -o {slurm_log_out}/change_detection_%a.out\n#SBATCH -e {slurm_log_error}/change_detection_%a.error\n#SBATCH --open-mode=append\n#SBATCH --job-name=ccdc\n#SBATCH --array=1-5000\n#SBATCH --mail-type=ALL\n#SBATCH --mail-user={slurm_email}\n#SBATCH --mem-per-cpu=24576\n\nsleep `shuf -i 1-120 -n 1`\n\nmodule load matlab/mcr-R2013a-v81\n\necho "\\n\\n********************************************"\necho "* This job runs on ${SLURM_JOB_NODELIST}"\necho "********************************************\\n\\n"\n\n# Set this to the last number of the array. New variable in Slurm 15.08 (coming soon)\nsetenv MATLAB_NTASKS 5000\n#setenv MATLAB_NTASKS ${SLURM_ARRAY_TASK_MAX}\nsetenv MATLAB_TASK ${SLURM_ARRAY_TASK_ID}\n\necho "MATLAB_NTASKS = ${MATLAB_NTASKS}"\necho "MATLAB_TASK = ${MATLAB_TASK}"\n\n\n# Running MATLAB standalone results in creation of a cache folder in home dir\n# This may 